In [ ]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from astropy.io import fits
import importlib

import statmorph
from statmorph.utils.image_diagnostics import make_figure
import utils.data
import utils.plots as plots
%matplotlib inline

In [ ]:
drz_file = 'cluster_data/abell2537/hlsp_relics_hst_wfc3ir-60mas_abell2537_f160w_v1_drz.fits'
cat_file = 'cluster_data/abell2537/hlsp_relics_hst_acs-wfc3ir_abell2537_multi_v1_cat.txt'
catalog = pd.read_csv(cat_file, comment='#', sep='\s+', index_col='id')
hdulist = fits.open(drz_file)
drz = hdulist[0].data

In [ ]:
catalog.head()

In [ ]:
galaxies = catalog[catalog['stel'] < 0.05 ]
galaxies.head()

In [ ]:
z_cluster = 0.2966
cluster_gals = galaxies[np.abs(galaxies['zb'] - z_cluster) < 0.15]
w = drz.shape[1]
h = drz.shape[0]
center = (int(w/2), int(h/2))
bcg = cluster_gals.loc[cluster_gals['area'].idxmax()]

In [ ]:
x = cluster_gals['x']
y = cluster_gals['y']
fig, axs = plt.subplots(figsize=(8,8))
plots.display_img(image=drz, axs=axs)
axs.scatter(center[0], center[1], marker='x', color='r')
axs.scatter(bcg['x'], bcg['y'], marker='o', color='r')
plt.show()

In [ ]:
cropped_image = utils.data.crop(drz, radius=1350)
fig, axs = plt.subplots(figsize=(8, 8))
plots.display_img(image=cropped_image, axs=axs)
plt.show()

In [ ]:
# full_segmap = segmentation_map(drz, nsigma=10.)
full_segmap = fits.open('cluster_data/abell1300/hlsp_relics_hst_acs-wfc3ir_abell1300_multi_v1_segm.fits')
full_segmap = full_segmap[0].data

In [ ]:
full_source_morphs = statmorph.source_morphology(image=drz,
                                                 segmap=full_segmap,
                                                 gain=2.25)

In [ ]:
def func(src):
    return src.r50      # sort on half-light radius to get BCG

full_source_morphs.sort(reverse=True, key=func)
bcg = full_source_morphs[1]
full_morph_df = create_morph_df(full_source_morphs)

In [ ]:
make_figure(bcg)

In [ ]:
zoomed_region, zoom_size = zoom_in(bcg, drz)
plot_light(bcg, zoomed_region, zoom_size)

In [ ]:
def zoom_in(object, radius, image):
    """
    Square image crop using statmorph source object properties.
    """
    zoom_size = 6 * radius
    x_min = int(object['x'] - zoom_size)
    x_max = int(object['x'] + zoom_size)
    y_min = int(object['y'] - zoom_size)
    y_max = int(object['y'] + zoom_size)

    # Extract the zoomed region
    zoomed_region = image[y_min:y_max, x_min:x_max]
    return zoomed_region, zoom_size

In [ ]:
radius = np.sqrt(bcg['area']/np.pi)
zoomed_region, zoom_size = zoom_in(bcg, radius, drz)

In [ ]:
fig, ax = plt.subplots()
ax.imshow(zoomed_region, cmap='gray', origin='lower', vmin=0, vmax=0.05)
for scale in np.linspace(1, 5, num=10):
    circle = Circle((zoom_size, zoom_size), radius*scale,
                            color='red', fill=False, linewidth=1.5)

    ax.add_patch(circle)
    
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.legend()
plt.show()

In [ ]:
center = (bcg['x'], bcg['y'])
radius = np.sqrt(bcg['area']/np.pi)
rads = radius*np.linspace(1, 3, num=5)
morphs_list = []
for radius in rads:
    mask = utils.data.circular_mask(drz, center, radius)
    # display_img(image=drz, mask=mask)
    sm_mask = mask.astype(int)
    morph = statmorph.source_morphology(drz,
                                     segmap=sm_mask,
                                     gain=2.25)
    morphs_list.append(morph[0])

In [ ]:
morphs_list

In [ ]:
def create_df(source_morphs, name=None, save=False):

    sources = []
    for src in source_morphs:
        sources.append({
            'xc_centroid': src.xc_centroid,
            'yc_centroid': src.yc_centroid,
            'ellipticity_centroid': src.ellipticity_centroid,
            'elongation_centroid': src.elongation_centroid,
            'orientation_centroid': src.orientation_centroid,
            'xc_asymmetry': src.xc_asymmetry,
            'yc_asymmetry': src.yc_asymmetry,
            'ellipticity_asymmetry': src.ellipticity_asymmetry,
            'elongation_asymmetry': src.elongation_asymmetry,
            'orientation_asymmetry': src.orientation_asymmetry,
            'rpetro_circ': src.rpetro_circ,
            'rpetro_ellip': src.rpetro_ellip,
            'rhalf_circ': src.rhalf_circ,
            'rhalf_ellip': src.rhalf_ellip,
            'r20': src.r20,
            'r50': src.r50,
            'r80': src.r80,
            'Gini': src.gini,
            'M20': src.m20,
            'F(G, M20)': src.gini_m20_bulge,
            'S(G, M20)': src.gini_m20_merger,
            'sn_per_pixel': src.sn_per_pixel,
            'C': src.concentration,
            'A': src.asymmetry,
            'S': src.smoothness,
            'sersic_amplitude': src.sersic_amplitude,
            'sersic_rhalf': src.sersic_rhalf,
            'sersic_n': src.sersic_n,
            'sersic_xc': src.sersic_xc,
            'sersic_yc': src.sersic_yc,
            'sersic_ellip': src.sersic_ellip,
            'sersic_theta': src.sersic_theta,
            'sersic_chi2_dof': src.sersic_chi2_dof,
            'sky_mean': src.sky_mean,
            'sky_median': src.sky_median,
            'sky_sigma': src.sky_sigma,
            'flag': src.flag,
            'flag_sersic': src.flag_sersic,
        })

    sources = pd.DataFrame(sources)
    if save:
        if name is not None:
            sources.to_csv(name)
        else:
            sources.to_csv('source_morphs.csv')
    return sources

In [ ]:
morph_df = create_df(morphs_list, save=True)

In [ ]:
morph_df.insert(0, 'Radius (px)', rads)

In [ ]:
morph_df.to_csv('abell2537_source_morphs.csv')

In [ ]:
mask = circular_mask(drz, center, rads[0])
display_img(image=drz, mask=mask)

mask = circular_mask(drz, center, rads[9])
display_img(image=drz, mask=mask)